In [1]:
catalog.list()


[
    'companies',
    'reviews',
    'shuttles',
    'preprocessed_companies',
    'preprocessed_shuttles',
    'model_input_table',
    'regressor',
    'scope3_data_3',
    'scope3_data_0',
    'preprocessed_scope3',
    'parameters',
    'params:feature_options',
    'params:feature_options.features',
    'params:model_options',
    'params:model_options.test_size',
    'params:model_options.random_state',
    'params:model_options.features'
]

In [10]:
import numpy as np
import pandas as pd
from typing import Dict

In [2]:
df = catalog.load("scope3_data_3")
df 

[09/18/23 14:53:08] INFO     Loading data from 'scope3_data_3' (ExcelDataSet)...                ]8;id=864499;file:///workspace/.pyenv_mirror/user/current/lib/python3.11/site-packages/kedro/io/data_catalog.py\data_catalog.py]8;;\:]8;id=285376;file:///workspace/.pyenv_mirror/user/current/lib/python3.11/site-packages/kedro/io/data_catalog.py#492\492]8;;\

,Year,Company Name,Country,Industry (Exiobase),GHG Intensity (Sales),GHG Intensity (Op Income),"Total GHG Environmental Impact (Scope 1, 2, 3)",Scope 1,Scope 2,Scope 3,...,Upstream Leased Assets Imputed,Upstream Transportation and Distribution,Upstream Transportation and Distribution Set,Upstream Transportation and Distribution Imputed,Use of Sold Products,Use of Sold Products Set,Use of Sold Products Imputed,Waste Generated in Operations,Waste Generated in Operations Set,Waste Generated in Operations Imputed
0,2020,REA GROUP LTD,AUSTRALIA,"Recreational, cultural and sporting activities...",-0.008351,-0.017857,-5.082181e+06,-7.967514e+01,-1.092801e+05,-4.972821e+06,...,0,-1.622937e+05,predicted,1,-2.517427e+05,predicted,1,-1.680007e+04,train,0
1,2020,SEEK LTD,AUSTRALIA,"Recreational, cultural and sporting activities...",-0.006768,-0.041618,-7.450531e+06,-4.174358e+05,-1.297567e+05,-6.903339e+06,...,0,-3.835789e+05,predicted,1,-4.218799e+06,predicted,1,-8.513858e+04,train,0
2,2020,SIMS LTD,UNITED STATES,Manufacture of basic iron and steel and of fer...,-0.190190,NaN,-6.454490e+08,-7.256229e+06,-7.571452e+06,-6.306213e+08,...,1,-4.439044e+06,predicted,1,-3.844370e+08,predicted,1,-8.435292e+07,train,0
3,2020,TRANSURBAN GROUP,AUSTRALIA,Other land transport,-0.039985,-0.218081,-9.978418e+07,-4.997908e+05,-1.585296e+07,-8.343143e+07,...,1,-9.207954e+05,predicted,1,-5.275860e+04,predicted,1,-2.550743e+05,test,0
4,2020,WORLEY LTD,AUSTRALIA,Extraction of crude petroleum and services rel...,-0.010303,-0.212981,-9.274720e+07,-6.682809e+06,-8.799892e+06,-7.726450e+07,...,1,-2.048789e+06,predicted,1,-4.953517e+07,predicted,1,-4.679207e+05,predicted,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9008,2012,CLICKS GROUP LTD,SOUTH AFRICA,"Retail trade, except of motor vehicles and mot...",-0.020833,-1.555053,-3.828618e+07,-2.161473e+05,-1.040865e+07,-2.766139e+07,...,1,-5.627341e+05,test,0,-1.586013e+07,predicted,1,-8.513858e+04,predicted,1
9009,2012,ARCELORMITTAL SOUTH AFRICA,SOUTH AFRICA,Manufacture of basic iron and steel and of fer...,-0.563217,NaN,-2.150594e+09,-1.288242e+09,-4.437372e+08,-4.186150e+08,...,1,-1.596656e+07,predicted,1,-2.102340e+08,predicted,1,-1.083696e+06,predicted,1
9010,2012,ROYAL BAFOKENG PLATINUM LTD,SOUTH AFRICA,Quarrying of sand and clay,-0.166875,-2.008172,-5.654073e+07,-3.797090e+05,-3.604515e+07,-2.011587e+07,...,1,-3.921155e+05,predicted,1,-2.190611e+05,predicted,1,-1.214363e+06,train,0
9011,2012,PPC LTD,SOUTH AFRICA,Quarrying of sand and clay,-1.076408,-4.237565,-9.514058e+08,-5.050641e+08,-6.762257e+07,-3.787191e+08,...,1,-8.957421e+05,predicted,1,-3.301965e+08,predicted,1,-8.637923e+04,predicted,1


In [3]:
features = catalog.load("params:feature_options.features")
features

[09/18/23 14:55:02] INFO     Loading data from 'params:feature_options.features'                ]8;id=68683;file:///workspace/.pyenv_mirror/user/current/lib/python3.11/site-packages/kedro/io/data_catalog.py\data_catalog.py]8;;\:]8;id=127444;file:///workspace/.pyenv_mirror/user/current/lib/python3.11/site-packages/kedro/io/data_catalog.py#492\492]8;;\
                             (MemoryDataset)...                                                                    


[
    'Country',
    'Industry (Exiobase)',
    'Business Travel',
    'Capital Goods',
    'Downstream Leased Assets',
    'Downstream Transportation and Distribution',
    'Employee Commuting',
    'End of Life Treatment of Sold Products',
    'Franchises',
    'Fuel-and-energy-related activities (not included in Scope 1 or 2)',
    'Investments',
    'Processing of Sold Products',
    'Purchased Goods and Services',
    'Upstream Leased Assets',
    'Upstream Transportation and Distribution',
    'Use of Sold Products',
    'Waste Generated in Operations'
]

In [5]:
pipelines


{'__default__': Pipeline([
Node(preprocess_companies, 'companies', 'preprocessed_companies', 'preprocess_companies_node'),
Node(preprocess_scope3, ['scope3_data_3', 'params:feature_options'], 'preprocessed_scope3', 'preprocess_scope3_node'),
Node(preprocess_shuttles, 'shuttles', 'preprocessed_shuttles', 'preprocess_shuttles_node'),
Node(create_model_input_table, ['preprocessed_shuttles', 'preprocessed_companies', 'reviews'], 'model_input_table', 'create_model_input_table_node'),
Node(split_data, ['model_input_table', 'params:model_options'], ['X_train', 'X_test', 'y_train', 'y_test'], 'split_data_node'),
Node(train_model, ['X_train', 'y_train'], 'regressor', 'train_model_node'),
Node(evaluate_model, ['regressor', 'X_test', 'y_test'], None, 'evaluate_model_node')
]), 'data_processing': Pipeline([
Node(preprocess_companies, 'companies', 'preprocessed_companies', 'preprocess_companies_node'),
Node(preprocess_scope3, ['scope3_data_3', 'params:feature_options'], 'preprocessed_scope3', 'pre

In [6]:
%run_viz

<IPython.core.display.HTML object>

In [11]:
def preprocess_scope3(scope3_data: pd.DataFrame, parameters: Dict) -> pd.DataFrame:
    """Preprocesses the Scope 3 data.

    Args:
        scope3_data: Raw data.
        
    Returns:
        Preprocessed data, with missing values removed.
    """
    
    features = scope3_data[parameters["features"]]
    preprocessed_data = _remove_missing_values(features)
    print("from notebook")
    return preprocessed_data